## Business Overview
Starting an organic food delivery service
what is the market like?
Organic food is the fastest-growing category in retail grocery today, but still represents just 6% of the total market share in the U.S. To grow the market, organic trade associations provide coupons and other advertising to incentivize purchases, but currently such promotions are not targeted to individual consumers.

Can likely organic buyers be identified in a way that grows organic sales?

Can we identify users who are likely to buy organic produce based on their past shopping history, combining recommendations from a collaborative filtering model with predictions from a logistic regression model.

The model identifies ___% new likely buyers of organic food and reduces spam to unlikely buyers by 40% compared to not targeting. More focused targeting will increase lift in the percentage of purchases with organic items.

## Data Overview
The dataset is anonymized and contains a sample of over 3 million grocery orders from more than 200,000 Instacart users. For each user, we provide between 4 and 100 of their orders, with the sequence of products purchased in each order.

### orders (3.4m rows, 206k users):

- order_id: order identifier
- user_id: customer identifier
- eval_set: which evaluation set this order belongs in (see SET described below)
- order_number: the order sequence number for this user (1 = first, n = nth)
- order_dow: the day of the week the order was placed on
- order_hour_of_day: the hour of the day the order was placed on
- days_since_prior: days since the last order, capped at 30 (with NAs for order_number = 1)

### products (50k rows):

- product_id: product identifier
- product_name: name of the product
- aisle_id: foreign key
- department_id: foreign key

### aisles (134 rows):

- aisle_id: aisle identifier
- aisle: the name of the aisle

### deptartments (21 rows):

- department_id: department identifier
- department: the name of the department

### order_products__SET (30m+ rows):

- order_id: foreign key
- product_id: foreign key
- add_to_cart_order: order in which each product was added to cart
- reordered: 1 if this product has been ordered by this user in the past, 0 otherwise

where SET is one of the four following evaluation sets (eval_set in orders):

- "prior": orders prior to that users most recent order (~3.2m orders)
- "train": training data supplied to participants (~131k orders)
- "test": test data reserved for machine learning competitions (~75k orders)

In [1]:
#imports
import pandas as pd
import numpy as np
import zipfile
import seaborn as sns
import matplotlib.pyplot as plt

#from category_encoders import OneHotEncoder

from sklearn.impute import SimpleImputer

from sklearn.model_selection import cross_val_score, train_test_split

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, roc_curve, roc_auc_score, log_loss, precision_score, recall_score, accuracy_score, f1_score

from sklearn.preprocessing import StandardScaler, PolynomialFeatures, normalize, LabelEncoder, MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

In [2]:
#read in files
aisles = pd.read_csv('data/aisles.csv.zip')
departments = pd.read_csv('data/departments.csv.zip')
orders = pd.read_csv('data/orders.csv.zip')
products = pd.read_csv('data/products.csv.zip')
order_products_train = pd.read_csv('data/order_products__train.csv.zip')
order_products_prior = pd.read_csv('data/order_products__prior.csv')

In [3]:
table_names = ['aisles','departments','orders','order_products_prior','order_products_train','products']
for i,df in enumerate([aisles,departments,orders,order_products_prior,order_products_train,products]):
    print(table_names[i])
    print(len(df))
    display(df.head())

aisles
134


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


departments
21


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


orders
3421083


...

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


order_products_train
1384617


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


products
49688


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [4]:
print(departments.values)
print(aisles.values)

[[1 'frozen']
 [2 'other']
 [3 'bakery']
 [4 'produce']
 [5 'alcohol']
 [6 'international']
 [7 'beverages']
 [8 'pets']
 [9 'dry goods pasta']
 [10 'bulk']
 [11 'personal care']
 [12 'meat seafood']
 [13 'pantry']
 [14 'breakfast']
 [15 'canned goods']
 [16 'dairy eggs']
 [17 'household']
 [18 'babies']
 [19 'snacks']
 [20 'deli']
 [21 'missing']]
[[1 'prepared soups salads']
 [2 'specialty cheeses']
 [3 'energy granola bars']
 [4 'instant foods']
 [5 'marinades meat preparation']
 [6 'other']
 [7 'packaged meat']
 [8 'bakery desserts']
 [9 'pasta sauce']
 [10 'kitchen supplies']
 [11 'cold flu allergy']
 [12 'fresh pasta']
 [13 'prepared meals']
 [14 'tofu meat alternatives']
 [15 'packaged seafood']
 [16 'fresh herbs']
 [17 'baking ingredients']
 [18 'bulk dried fruits vegetables']
 [19 'oils vinegars']
 [20 'oral hygiene']
 [21 'packaged cheese']
 [22 'hair care']
 [23 'popcorn jerky']
 [24 'fresh fruits']
 [25 'soap']
 [26 'coffee']
 [27 'beers coolers']
 [28 'red wines']
 [29 'ho

In [5]:
#add a column for whether an item is organic or not:   
def return_organic(prod_name):
    words = prod_name.lower().split()
    if 'organic' in words:
        return 1
    else:
        return 0
    
products['organic'] = products['product_name'].map(return_organic)

#also merge aisle and department names
products = products.merge(aisles,on='aisle_id')
products = products.merge(departments,on='department_id')
products.loc[:,'organic_produce'] = 0
products.loc[(products['organic']==1) & (products['department']=='produce') ,
         'organic_produce'] = 1

In [6]:
# Merge products_df with order_products__train on product_id
order_products = pd.merge(products, order_products_train, on='product_id')

# Merge order_products with orders on order_id
order_df = pd.merge(order_products, orders, on='order_id')

In [7]:
organic_df = order_df[(order_df['organic'] == 1)]
organic_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 403331 entries, 4 to 1384608
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   product_id              403331 non-null  int64  
 1   product_name            403331 non-null  object 
 2   aisle_id                403331 non-null  int64  
 3   department_id           403331 non-null  int64  
 4   organic                 403331 non-null  int64  
 5   aisle                   403331 non-null  object 
 6   department              403331 non-null  object 
 7   organic_produce         403331 non-null  int64  
 8   order_id                403331 non-null  int64  
 9   add_to_cart_order       403331 non-null  int64  
 10  reordered               403331 non-null  int64  
 11  user_id                 403331 non-null  int64  
 12  eval_set                403331 non-null  object 
 13  order_number            403331 non-null  int64  
 14  order_dow          

In [8]:
organic_df.head()

,product_id,product_name,aisle_id,department_id,organic,aisle,department,organic_produce,order_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
4,37600,Organic Kettle Corn,23,19,1,popcorn jerky,snacks,0,6695,1,1,1540,train,29,1,1,8.0
9,12542,Organic Creamy Almond Butter,88,13,1,spreads,pantry,0,6695,8,1,1540,train,29,1,1,8.0
10,26317,Organic Lemonade,98,7,1,juice nectars,beverages,0,6695,11,1,1540,train,29,1,1,8.0
19,30777,Organic Classic Hummus,67,20,1,fresh dips tapenades,deli,0,6695,15,1,1540,train,29,1,1,8.0
30,49235,Organic Half & Half,53,16,1,cream,dairy eggs,0,48361,1,1,194636,train,5,2,14,11.0


In [9]:
organic_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 403331 entries, 4 to 1384608
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   product_id              403331 non-null  int64  
 1   product_name            403331 non-null  object 
 2   aisle_id                403331 non-null  int64  
 3   department_id           403331 non-null  int64  
 4   organic                 403331 non-null  int64  
 5   aisle                   403331 non-null  object 
 6   department              403331 non-null  object 
 7   organic_produce         403331 non-null  int64  
 8   order_id                403331 non-null  int64  
 9   add_to_cart_order       403331 non-null  int64  
 10  reordered               403331 non-null  int64  
 11  user_id                 403331 non-null  int64  
 12  eval_set                403331 non-null  object 
 13  order_number            403331 non-null  int64  
 14  order_dow          

In [10]:
organic_df['is_weekend'] = organic_df['order_dow'].map(lambda d: 1 if d >= 5 else 0)
organic_df['day_period'] = pd.cut(organic_df['order_hour_of_day'], 
                                  bins=[-1, 11, 16, 23], 
                                  labels=['Morning', 'Afternoon', 'Night'], 
                                  include_lowest=True)

In [11]:
organic_df.to_csv('organic_data.csv', index=True)

In [12]:
organic_df = organic_df[[
    'product_id',
    'product_name',
    'aisle_id',
    'aisle',
    'department_id',
    'department',
    'user_id',
    'days_since_prior_order',
    'reordered',
    'order_id',
    'order_number',
    'add_to_cart_order',
    'order_hour_of_day',
    'day_period',
    'is_weekend',
    'organic',
    'organic_produce',
    'eval_set'
]]

In [13]:
filtered_departments = ['pets', 'household',]
organic_df = organic_df[~organic_df['department'].isin(filtered_departments)]

In [14]:
organic_df['department'].value_counts()

produce            210386
dairy eggs          60932
canned goods        20529
snacks              18030
frozen              16980
pantry              16635
beverages           12596
dry goods pasta      9354
bakery               8177
deli                 7047
babies               5937
breakfast            5558
meat seafood         4539
missing              3204
international        1705
personal care         913
bulk                  630
other                  92
alcohol                24
Name: department, dtype: int64

In [15]:

display(organic_df.loc[organic_df['organic']==1,['product_name','department']].value_counts().head(60))


product_name                                                    department  
Bag of Organic Bananas                                          produce         15480
Organic Strawberries                                            produce         10894
Organic Baby Spinach                                            produce          9784
Organic Avocado                                                 produce          7409
Organic Hass Avocado                                            produce          7293
Organic Raspberries                                             produce          5546
Organic Blueberries                                             produce          4966
Organic Whole Milk                                              dairy eggs       4908
Organic Cucumber                                                produce          4613
Organic Zucchini                                                produce          4589
Organic Yellow Onion                                           

In [16]:
# Update rows containing 'beef' or 'chicken' in the product name to 'meat seafood'
organic_df.loc[organic_df['product_name'].str.contains('beef|chicken', case=False), 'department'] = 'meat seafood'

# Update rows containing 'yogurt' in the product name to 'dairy eggs'
organic_df.loc[organic_df['product_name'].str.contains('yogurt|heavy cream|cottage cheese|coconut cream|coconut milk|cheddar cheese', case=False), 'department'] = 'dairy eggs'

# Update rows containing 'pasta' in the product name to 'dry goods pasta'
organic_df.loc[organic_df['product_name'].str.contains('pasta|jasmine|brown rice|fettuccine', case=False), 'department'] = 'dry goods pasta'

organic_df.loc[organic_df['product_name'].str.contains('cauliflower|cookie dough', case=False), 'department'] = 'frozen'

organic_df.loc[organic_df['product_name'].str.contains('juice|drink|water|hot chocolate|thirst|almond milk|soy milk|smoothie|kombucha|tonic|tea|Non-Dairy Beverage', case=False), 'department'] = 'beverage'

organic_df.loc[organic_df['product_name'].str.contains('chips|kettle corn|cookies|snack|leather|dippers|chocolate spread|crackers', case=False), 'department'] = 'snacks'

organic_df.loc[organic_df['product_name'].str.contains('bread|corn tortillas', case=False), 'department'] = 'bakery'

organic_df.loc[organic_df['product_name'].str.contains('hummus|spread', case=False), 'department'] = 'deli'

organic_df.loc[organic_df['product_name'].str.contains('cinnamon rolls|Crescent|protein', case=False), 'department'] = 'breakfast'

organic_df.loc[organic_df['product_name'].str.contains('flour|husk|almonds|granola|rice|oats', case=False), 'department'] = 'pantry'

organic_df.loc[organic_df['product_name'].str.contains('babyfood|puree', case=False), 'department'] = 'babies'

In [17]:
produce_name = ['apple','carrot','celery','jicama','salad','cucumber','pepper','kale','peach','squash','mango',
                'lime','beet','kiwi','yam','potato','pear','orange','broccoli','raddish','bunch','eggplant','nectarine',
               'fennel','citrus','kit','sweet corn']
organic_df.loc[organic_df['product_name'].str.contains('|'.join(produce_name), case=False), 'department'] = 'produce'

In [18]:
organic_df = organic_df[organic_df['department'] != 'missing']

In [19]:
organic_df.tail(60)

,product_id,product_name,aisle_id,aisle,department_id,department,user_id,days_since_prior_order,reordered,order_id,order_number,add_to_cart_order,order_hour_of_day,day_period,is_weekend,organic,organic_produce,eval_set
1383020,29871,Organic Roast Beef,96,lunch meat,20,meat seafood,155016,30.0,1,1765737,4,1,14,Afternoon,0,1,0,train
1383021,29871,Organic Roast Beef,96,lunch meat,20,meat seafood,77170,17.0,0,3084361,5,1,14,Afternoon,1,1,0,train
1383058,40463,Organic Roasted Garlic Hummus,67,fresh dips tapenades,20,deli,67230,30.0,1,7865,13,2,10,Morning,1,1,0,train
1383080,26856,Organic Ezekiel 4:9 Sesame Bread,112,bread,3,bakery,7379,6.0,1,1707403,5,2,9,Morning,0,1,0,train
1383088,18923,Organic Minestrone Soup,1,prepared soups salads,20,deli,181612,30.0,1,3206663,5,2,18,Night,0,1,0,train
1383098,4162,Organic Thai Coconut Curry Hummus,67,fresh dips tapenades,20,deli,75049,10.0,0,1898548,23,1,11,Morning,0,1,0,train
1383109,11069,Organic Kale Pesto Hummus,67,fresh dips tapenades,20,produce,55122,6.0,1,2170373,39,2,15,Afternoon,0,1,0,train
1383111,11871,Organic Cranberry Sauce,67,fresh dips tapenades,20,deli,44341,3.0,0,264305,9,3,12,Afternoon,0,1,0,train
1383135,18615,Organic Guacamole,67,fresh dips tapenades,20,deli,154127,4.0,0,554457,7,2,16,Afternoon,0,1,0,train
1383136,30777,Organic Classic Hummus,67,fresh dips tapenades,20,deli,154127,4.0,1,554457,7,1,16,Afternoon,0,1,0,train


In [20]:
missing_aisle_df = organic_df[organic_df['aisle'] == 'missing'][['product_name', 'aisle', 'department']]

In [21]:
missing_aisle_df.shape

(3195, 3)

In [22]:
missing_aisle_df.head(60)

,product_name,aisle,department
3087,Organic Ground Chicken,missing,meat seafood
3216,Organic Strawberry Fruit Leather,missing,snacks
3318,Organic Whole Milk Washington Black Cherry Yogurt,missing,dairy eggs
3694,Organic Uncured Beef Hot Dog,missing,meat seafood
4139,Organic English Seedless Cucumber,missing,produce
4463,Apple Frank Organic Beef,missing,produce
4577,Organic Whole Milk Washington Black Cherry Yogurt,missing,dairy eggs
5648,Organic Classic Original Hummus,missing,deli
5809,Organic Mango Yogurt,missing,produce
6635,Organic Nondairy Lemon Cashew Yogurt,missing,dairy eggs


In [23]:
organic_df.loc[organic_df['product_name'].str.contains('hummus', case=False),'aisle'] = 'dips'

In [24]:
products = organic_df[organic_df['product_name'].str.contains('hummus', case=False)]
products[['product_name', 'aisle']].head(60)

,product_name,aisle
19,Organic Classic Hummus,dips
654,Organic Hummus,dips
1846,Organic Original Hummus,dips
3756,Organic Hummus,dips
5648,Organic Classic Original Hummus,dips
6588,Organic Thai Coconut Curry Hummus,dips
6636,Organic Traditional Hummus,dips
8609,Organic Traditional Hummus,dips
9131,Organic Jalapeno Cilantro Hummus,dips
12410,Organic Roasted Red Pepper Hummus,dips


In [27]:
organic_2 = pd.crosstab(organic_df['order_id'], organic_df['product_name'])
organic_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93121 entries, 1 to 3421070
Columns: 4108 entries, 0% Fat Free Organic Milk to Zucchini Banana & Amaranth Organic Baby Food
dtypes: int64(4108)
memory usage: 2.9 GB
